<a href="https://colab.research.google.com/github/ambern-nguyen/Air-Quality-Prediction/blob/main/Air_Quality_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Group 4 Final Project: Air Quality Prediction

# Importing Libraries

In [ ]:
!pip install torcheval
!pip install pynvml
!pip install psutil
import time
import torch
import pynvml
from pynvml import *
import psutil
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torcheval.metrics import R2Score

# Following command should only be run on T4 GPU Runtime
#nvmlInit()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 3.5 MB/s eta 0:00:00


# Importing CSV and Creating Usable Tensor

Not only will we need to use pandas to read the CSV, we will also have to manually change some things with formatting to ensure that the transformation into a tensor is possible.

In [ ]:
# Create pandas dataframe from CSV
df = pd.read_csv("AirQualityUCI.csv", delimiter=";")

# Replace instances of "," with "." - we believe this is a european thing
df = df.replace(",", ".", regex=True)

# Remove attributes from dataframe we do not need
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")

del df["Unnamed: 15"]
del df["Unnamed: 16"]
del df["Date"]
del df["Time"]

# Convert object attributes to float64

df["CO(GT)"] = pd.to_numeric(df["CO(GT)"], errors="coerce")
df["C6H6(GT)"] = pd.to_numeric(df["C6H6(GT)"], errors="coerce")
df["T"] = pd.to_numeric(df["T"], errors="coerce")
df["RH"] = pd.to_numeric(df["RH"], errors="coerce")
df["AH"] = pd.to_numeric(df["AH"], errors="coerce")
df = df[~df.isin([-200]).any(axis=1)]


# Remove entries with NaN
df = df.dropna()

# Create tensor from dataframe
datatensor = torch.tensor(df.values, dtype=torch.float32)

# Defining function to get AQI from Sensor Readings

This function will accept two inputs for CO and NO2 in the air for a given hour, and output the AQI based on our research of it's formula.

In this way, this function will be used as the post-output layer of our model, condensing predicted values into a single "Predicted" AQI.

Conversely, this function can be run on the Ground Truth values in the dataset to get the "True" value of AQI.

This comparison will be what leads us to the ultimate R^2 value we will use to grade our model's accuracy.

In [ ]:
import torch

def calc_individual_aqi(conc, breakpoints):
    for C_lo, C_hi, I_lo, I_hi in breakpoints:
        if C_lo <= conc <= C_hi:
            return ((I_hi - I_lo) / (C_hi - C_lo)) * (conc - C_lo) + I_lo
    return 0  # return 0 if calculation out of range

def ugm3_to_ppm(co_ugm3, temp, molecular_weight=28.01, pressure_kpa=101.325):
    # Convert CO from micrograms/m³ to ppm.
    temp_k = temp + 273.15  # Convert °C to Kelvin
    ppm = (co_ugm3 * temp_k * 8.314) / (molecular_weight * pressure_kpa * 1000)
    return ppm

def ugm3_to_ppb(no2_ugm3, temp, molecular_weight=46.0055, pressure_kpa=101.325):
    # Convert NO2 from micrograms/m³ to ppb.
    temp_k = temp + 273.15
    ppb = (no2_ugm3 * temp_k * 8.314) / (molecular_weight * pressure_kpa)
    return ppb

def get_AQI(predictions, temp, pressure_kpa=101.325):
    CO_breakpoints = [
        (0.0, 4.4, 0, 50),
        (4.5, 9.4, 51, 100),
        (9.5, 12.4, 101, 150),
        (12.5, 15.4, 151, 200),
        (15.5, 30.4, 201, 300)
    ]

    NO2_breakpoints = [
        (0, 53, 0, 50),
        (54, 100, 51, 100),
        (101, 360, 101, 150),
        (361, 649, 151, 200),
        (650, 1249, 201, 300)
    ]

    co = predictions[:, 0].cpu().numpy()
    no2 = predictions[:, 1].cpu().numpy()

    aqi_list = []
    for i in range(len(co)):
        co_ppm = ugm3_to_ppm(co[i], temp, pressure_kpa=pressure_kpa)
        no2_ppb = ugm3_to_ppb(no2[i], temp, pressure_kpa=pressure_kpa)

        aqi_co = calc_individual_aqi(co_ppm, CO_breakpoints)
        aqi_no2 = calc_individual_aqi(no2_ppb, NO2_breakpoints)
        aqi = max(aqi_co, aqi_no2)
        aqi_list.append(aqi)

    return torch.tensor(aqi_list, dtype=torch.float32)


# Creating Feed Forward Model with ReLU Activation

Layers are as follows:

Input Layer: Accepts 8 inputs and uses feature mapping to output 64 neurons.

Hidden Layer: Accepts 64 input neurons to output 64 neurons for higher precision.

Output Layer: Accepts 64 input neurons to predict two values: CO and NO2.

In [ ]:
# Establish device
device = "cpu"

class AirQualityPredictor(nn.Module):
  # Define layers
  def __init__(self):
    super(AirQualityPredictor, self).__init__()
    # Input layer: 8 inputs to 64 neurons

    self.IL = nn.Linear(8, 64)
    # Hidden Layer: 64 inputs to 64 neurons - for parsing more complex features

    self.HL = nn.Linear(64, 64)

    # Output Layer:  predicting CO and NO2 ground truth values
    self.OL = nn.Linear(64, 2) # Changed output size to 2 to match the number of target features


    # ReLU layer to be invoked during forward
    self.relu = nn.ReLU()

    # Batch normalization before ReLU
    self.bn1 = nn.BatchNorm1d(64)
    self.bn2 = nn.BatchNorm1d(64)


  # Define forward pass which accepts inputs, running relu between each layer
  def forward(self, inputs):
    x = self.IL(inputs)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.HL(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.OL(x)
    return x

# Status Check functions for CPU / GPU

In [ ]:
def get_CPU_status():
  print("CPU Usage:", psutil.cpu_percent(), "%")

def get_current_energy():
  pynvml.nvmlInit()
  handle = pynvml.nvmlDeviceGetHandleByIndex(0)
  gpuEnergyUsage = pynvml.nvmlDeviceGetTotalEnergyConsumption(handle)
  return gpuEnergyUsage

def get_GPU_status():
  # Test for non-quantized model
  handle = pynvml.nvmlDeviceGetHandleByIndex(0)
  startEnergy = get_current_energy()
  startTime = time.time()
  endEnergy = get_current_energy()
  endTime = time.time()
  print("Energy consumed:", endEnergy - startEnergy)
  print("Runtime:", endTime - startTime)
  print("Power used in Watts:", pynvml.nvmlDeviceGetPowerUsage(handle) / 1000)
  print("GPU Temperature:",  pynvml.nvmlDeviceGetTemperature(handle, 0))
  print("GPU Usage:", psutil.virtual_memory().percent, "%")


# Parse Dataset into Training / Testing Subsets

In [ ]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

input_cols = ['PT08.S1(CO)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)', 'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH']
target_cols = ['CO(GT)', 'NO2(GT)']

# Extract X and y from the dataframe
X = df[input_cols]

y = df[target_cols]


data = pd.concat([X, y], axis=1).dropna()  # Now X and y are defined

X = data[input_cols]
y = data[target_cols]

# Normalize inputs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

target_scaler = StandardScaler()
y_scaled = target_scaler.fit_transform(y)

# Convert to tensors
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)  # now shape [n_samples, 2]

X_tensor = X_tensor.to(device)
y_tensor = y_tensor.to(device)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

# Create TensorDataset and DataLoader
train_ds = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

# Invoke Model and Perform Training Loop

In [ ]:
model = AirQualityPredictor()
model = model.to(device)

# With Adam optimizer
def cpu_training():

  # Source loss fn and optimizer
  lossFn = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)

  # Source scaler for data pre-processing
  scaler = StandardScaler()

  # Print cpu frequency
  # print("CPU Frequency:", psutil.cpu_freq().current, "MHz")

  startTime = time.time()
  # Training loop
  num_epochs = 500
  for epoch in range(num_epochs):
    # Iterate over training data
      for batch_X, batch_y in train_loader:

          batch_X = batch_X.to(device)
          batch_y = batch_y.to(device)

          # Swap to training mode
          model.train()

          # Forward pass
          outputs = model(batch_X)
          loss = lossFn(outputs, batch_y)

          # Backward pass and optimization
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      # Print loss every 10 epochs
      if (epoch + 1) % 10 == 0:
          print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
  endTime = time.time()
  print("Time taken:", endTime - startTime)


# With SGD Optimizer
def gpu_training():

  # Source loss fn and optimizer
  lossFn = nn.MSELoss()
  optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

  # Source scaler for data pre-processing
  scaler = StandardScaler()

  startTime = time.time()
  # Training loop
  num_epochs = 500
  for epoch in range(num_epochs):
    # Iterate over training data
      for batch_X, batch_y in train_loader:

          batch_X = batch_X.to(device)
          batch_y = batch_y.to(device)

          # Swap to training mode
          model.train()

          # Forward pass
          outputs = model(batch_X)
          loss = lossFn(outputs, batch_y)

          # Backward pass and optimization
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      # Print loss every 10 epochs
      if (epoch + 1) % 10 == 0:
          print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
          print()
  endTime = time.time()
  print("Time taken:", endTime - startTime)

In [ ]:
cpu_training()

Epoch [10/500], Loss: 3234.0996
Epoch [20/500], Loss: 3236.8687
Epoch [30/500], Loss: 1030.7642
Epoch [40/500], Loss: 161.4977
Epoch [50/500], Loss: 94.6965
Epoch [60/500], Loss: 154.8145
Epoch [70/500], Loss: 63.1213
Epoch [80/500], Loss: 61.2131
Epoch [90/500], Loss: 65.2098
Epoch [100/500], Loss: 170.5101
Epoch [110/500], Loss: 87.0590
Epoch [120/500], Loss: 48.3215
Epoch [130/500], Loss: 70.0621
Epoch [140/500], Loss: 40.2340
Epoch [150/500], Loss: 89.6506
Epoch [160/500], Loss: 34.3354
Epoch [170/500], Loss: 130.0718
Epoch [180/500], Loss: 73.9185
Epoch [190/500], Loss: 87.4219
Epoch [200/500], Loss: 52.1951
Epoch [210/500], Loss: 32.4320
Epoch [220/500], Loss: 98.0988
Epoch [230/500], Loss: 38.2058
Epoch [240/500], Loss: 129.7100
Epoch [250/500], Loss: 47.0099
Epoch [260/500], Loss: 55.3440
Epoch [270/500], Loss: 83.7461
Epoch [280/500], Loss: 76.4905
Epoch [290/500], Loss: 59.7378
Epoch [300/500], Loss: 54.3344
Epoch [310/500], Loss: 28.3669
Epoch [320/500], Loss: 70.5821
Epoch 

In [ ]:
gpu_training()

Epoch [10/500], Loss: 43.5262

Epoch [20/500], Loss: 64.5575

Epoch [30/500], Loss: 70.7411

Epoch [40/500], Loss: 73.0862

Epoch [50/500], Loss: 123.4236

Epoch [60/500], Loss: 198.1784

Epoch [70/500], Loss: 73.4575

Epoch [80/500], Loss: 323.9131

Epoch [90/500], Loss: 113.9480

Epoch [100/500], Loss: 67.7167

Epoch [110/500], Loss: 261.7369

Epoch [120/500], Loss: 128.1791

Epoch [130/500], Loss: 17.6309

Epoch [140/500], Loss: 347.4381

Epoch [150/500], Loss: 72.7840

Epoch [160/500], Loss: 49.6034

Epoch [170/500], Loss: 45.0724

Epoch [180/500], Loss: 84.7604

Epoch [190/500], Loss: 260.6116

Epoch [200/500], Loss: 59.6397

Epoch [210/500], Loss: 136.9796

Epoch [220/500], Loss: 87.8196

Epoch [230/500], Loss: 58.0127

Epoch [240/500], Loss: 40.8056

Epoch [250/500], Loss: 125.7147

Epoch [260/500], Loss: 132.9351

Epoch [270/500], Loss: 74.6440

Epoch [280/500], Loss: 39.1575

Epoch [290/500], Loss: 93.7267

Epoch [300/500], Loss: 36.8582

Epoch [310/500], Loss: 135.1819

Epoch

# Final Test to Evaluate CPU and GPU Status and R^2 Values for CO and NO2 prediction
This shows the CPU and GPU usage from training the model as well as the R^2 values for CO and NO2. It also shows the R^2 values for CO and NO2 as well as the average R^2 from the two.

In [ ]:
# Run for CPU - COMMENT OUT WHEN NOT USING
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    outputs = outputs.to(device)
    r2_score_co = R2Score().to(device)
    r2_score_no2 = R2Score().to(device)
    r2_score_co.update(outputs[:, 0], y_test[:, 0])
    r2_score_no2.update(outputs[:, 1], y_test[:, 1])
    get_CPU_status()
    cpu_freq = psutil.cpu_freq()
    print(f'CPU Frequency: {cpu_freq}')
    print(f'R^2 Score for CO: {r2_score_co.compute()}')
    print(f'R^2 Score for NO2: {r2_score_no2.compute()}')
    print(f'Average R^2 Score: {(r2_score_no2.compute() + r2_score_co.compute())/2}')


CPU Usage: 34.0 %
CPU Frequency: scpufreq(current=2199.998, min=0.0, max=0.0)
R^2 Score for CO: 0.9513370394706726
R^2 Score for NO2: 0.8996989130973816
Average R^2 Score: 0.9255179762840271


In [ ]:
# Run for GPU - COMMENT OUT WHEN NOT USING
# model.eval()
# with torch.no_grad():
#     outputs = model(X_test)
#     outputs = outputs.to(device)
#     r2_score_co = R2Score().to(device)
#     r2_score_no2 = R2Score().to(device)
#     r2_score_co.update(outputs[:, 0], y_test[:, 0])
#     r2_score_no2.update(outputs[:, 1], y_test[:, 1])
#     get_GPU_status()
#     print(f'R^2 Score for CO: {r2_score_co.compute()}')
#     print(f'R^2 Score for NO2: {r2_score_no2.compute()}')
#     print(f'Average R^2 Score: {(r2_score_no2.compute() + r2_score_co.compute())/2}')


# Create Table with Only Model Inputs
These lines of code create a copy of the dataframe and leave only the inputs for the model so that the dataframe can be easily sampled and input into the model to get predicted values for CO and NO2 values. These values can then be put into the get_AQI function to find the predicted AQI based off of the CO and NO2 values given by the model.

In [ ]:
df_sample_inputs = df.copy()
columns_to_drop = ['CO(GT)', 'NMHC(GT)', 'C6H6(GT)', 'NOx(GT)', 'NO2(GT)']
df_sample_inputs = df.drop(columns=columns_to_drop)
df_sample_inputs

,PT08.S1(CO),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,1360.0,1046.0,1056.0,1692.0,1268.0,13.6,48.9,0.7578
1,1292.0,955.0,1174.0,1559.0,972.0,13.3,47.7,0.7255
2,1402.0,939.0,1140.0,1555.0,1074.0,11.9,54.0,0.7502
3,1376.0,948.0,1092.0,1584.0,1203.0,11.0,60.0,0.7867
4,1272.0,836.0,1205.0,1490.0,1110.0,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...
1226,1449.0,1282.0,625.0,2100.0,1569.0,19.1,61.1,1.3345
1227,1363.0,1152.0,684.0,1951.0,1495.0,18.2,65.4,1.3529
1228,1371.0,1136.0,689.0,1927.0,1471.0,18.1,66.1,1.3579
1229,1406.0,1107.0,718.0,1872.0,1384.0,17.7,66.9,1.3422


# Create Table with Only Ground Truth Values
These lines of code create a table that only hold the ground truth values for CO and NO2. These are used to get the ground truth AQI value which can then be compared to the predicted AQI value to find the accuracy of the model.

In [ ]:
df_ground_truth_AQI = df.copy()
columns_to_drop = ['PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'PT08.S4(NO2)', 'PT08.S5(O3)', 'RH', 'AH']
df_ground_truth_AQI = df.drop(columns=columns_to_drop)
df_ground_truth_AQI

,CO(GT),NO2(GT),T
0,2.6,113.0,13.6
1,2.0,92.0,13.3
2,2.2,114.0,11.9
3,2.2,122.0,11.0
4,1.6,116.0,11.2
...,...,...,...
1226,4.4,133.0,19.1
1227,3.1,110.0,18.2
1228,3.0,102.0,18.1
1229,3.1,108.0,17.7


# Random Test Row Selection
This function is used to randomly select a row from the dataframe which only stores the values needed to be input into the model. It returns the values in that row as well as the index of the row so that the ground truth values can be found later.

In [ ]:
import random
import torch

def get_random_inputs(df):
  if df.empty:
    return None

  random_index = random.randint(0, len(df) - 1)
  random_row = df.iloc[random_index]
  tensor_row = torch.tensor(random_row.values, dtype=torch.float32).to(device)
  tensor_row = tensor_row.unsqueeze(0)
  return tensor_row, random_index


# Random Testing
This randomly selects rows and gets a model prediction for each randomly selected input row. The model's predicted CO and NO2 values are plugged into the get_AQI function to find the predicted AQI. The ground truth values from the table are then used to find the ground truth AQI.

In [ ]:
predicted_aqi = []
ground_truth_aqi = []
r2_score = R2Score()

for i in range(0,500,16):
  sample_input, row_num = get_random_inputs(df_sample_inputs)

  # 4. Disable gradient tracking (not training)
  with torch.no_grad():
      outputs = model(sample_input)

  gt_row = df_ground_truth_AQI.iloc[row_num]
  gt_row_tensor = torch.tensor(gt_row.values, dtype=torch.float32).to(device)
  gt_row_tensor = gt_row_tensor.unsqueeze(0)

  AQI_predict = get_AQI(outputs, gt_row['T'])

  print("Model output (predicted CO and NO2):", outputs)

  print("Ground Truth AQI:", get_AQI(gt_row_tensor, gt_row['T']))

  # 5. Print outputs (predicted CO and NO2 values)
  print("AQI:", AQI_predict)
  print("\n")

  # Get ground truth and predicted AQI and append to the respective arrays
  AQI_predict_tensor = AQI_predict.to(device)
  ground_truth_aqi_tensor = get_AQI(gt_row_tensor, gt_row['T']).to(device)

  predicted_aqi.append(AQI_predict_tensor.item())
  ground_truth_aqi.append(ground_truth_aqi_tensor.item())

  r2_score.update(AQI_predict_tensor, ground_truth_aqi_tensor)

# Compute and print out the R^2 value for the AQI for the model
final_r2_score = r2_score.compute()
print("R^2 Score:", final_r2_score)

Model output (predicted CO and NO2): tensor([[ 1.6193, 79.1719]])
Ground Truth AQI: tensor([36.9372])
AQI: tensor([38.4788])


Model output (predicted CO and NO2): tensor([[ 0.8219, 77.8287]])
Ground Truth AQI: tensor([36.6047])
AQI: tensor([37.4855])


Model output (predicted CO and NO2): tensor([[  3.4550, 129.4763]])
Ground Truth AQI: tensor([62.0751])
AQI: tensor([64.5316])


Model output (predicted CO and NO2): tensor([[ 1.5443, 84.0072]])
Ground Truth AQI: tensor([39.3809])
AQI: tensor([40.8429])


Model output (predicted CO and NO2): tensor([[  3.0872, 126.8374]])
Ground Truth AQI: tensor([60.5007])
AQI: tensor([64.9148])


Model output (predicted CO and NO2): tensor([[ 0.7004, 48.6128]])
Ground Truth AQI: tensor([21.5772])
AQI: tensor([23.8393])


Model output (predicted CO and NO2): tensor([[  4.5953, 160.5264]])
Ground Truth AQI: tensor([86.7220])
AQI: tensor([84.1939])


Model output (predicted CO and NO2): tensor([[ 0.8746, 64.4661]])
Ground Truth AQI: tensor([31.1911])
AQI